In [1]:
import pickle
with open('asin_list.pickle', 'rb') as f:
    asin_list = pickle.load(f)

len(asin_list)

1831

In [7]:
import glob, os, pickle
product_list = []

os.chdir(r"H:\w_crawler\Amazon China Food\data")
for file in glob.glob("*.pickle"):
    with open(file, 'rb') as f:
        item = pickle.load(f)
        product_list.extend(item)
        print(file,len(item))

product_dict = {}
for prod in product_list:
    try:
        product_dict[prod['ASIN']] = prod
    except:
        print(prod)

len(product_dict)

休闲零食.pickle 3
冲调.pickle 3
南北干货.pickle 3
咖啡及可可.pickle 3
方便食品.pickle 3
水饮、乳品、果汁.pickle 3
烘焙.pickle 3
米面杂粮.pickle 3
糕点饼干.pickle 3
糖果、巧克力.pickle 3
节日、礼盒、礼券.pickle 3
茶.pickle 3
调味佐料.pickle 3
酒.pickle 3
食用油.pickle 3
鲜花.pickle 3
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data
category_name
category_url
data


0

In [8]:
with open("H:\w_crawler\Amazon China Food\data\糕点饼干.pickle", 'rb') as f:
    item = pickle.load(f)

In [3]:
asin_list = list(set(asin_list) - set(product_dict.keys()))
len(asin_list)

42

In [4]:
with open(r'H:\Amazon China Food\asin_list.pickle', 'wb') as f:
    pickle.dump(asin_list, f)

with open(r'H:\Amazon China Food\prod_list_2k.pickle', 'wb') as f:
    pickle.dump(product_dict, f)


In [5]:
import glob, os
product_dict = {}

os.chdir(r"H:\Amazon China Food\product data\prod_dict")
for file in glob.glob("*.pickle"):
    with open(file, 'rb') as f:
        item = pickle.load(f)
        product_dict.update(item)
        print(file,len(item))

len(product_dict)

prod_list_10k.pickle 9895
prod_list_11k.pickle 11603
prod_list_2k.pickle 1796
prod_list_8k.pickle 7993
prod_list_9k.pickle 9422


40686

In [6]:
import pandas as pd

df = pd.read_csv("H:\Amazon China Food\data\Amazon_China_Food.csv")

In [7]:
df.product_id.describe()

count          46697
unique         40712
top       B0116U7ZYG
freq               4
Name: product_id, dtype: object

In [9]:
def get_manu (prod_id):
    try:
        return product_dict.get(prod_id,None).get("制造商",None)
    except:
        return None

In [10]:
df['Manufacturer'] = df['product_id'].apply(lambda x: get_manu(x))

In [12]:
df.to_csv("H:\Amazon China Food\Amazon_China_Food.csv",index=None)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("H:\w_crawler\Amazon China Food\data\Amazon_China_Food.csv")

In [9]:
products = list(df['product_id'].unique())

In [29]:
import re
def Clean(text):
    text = text.replace("\n","").replace("\t","").replace("\r","")
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

In [31]:
import requests,random
response = requests.get(f"https://proxy.webshare.io/api/v2/proxy/list/?mode=direct&page={random.randint(1,5)}&page_size=50", headers={"Authorization": "94a3056cb6e4d13ef5b61adbb44c8b99d2f9aef4"})
data_dict = response.json()
proxy_list = list(map(lambda x: f"http://{x['username']}:{x['password']}@{x['proxy_address']}:{x['port']}",data_dict['results']))

In [35]:
import lxml.html
import requests

headers = {
    'Accept': 'text/html,*/*',
    'Accept-Language': 'en-US,en;q=0.9,ja;q=0.8',
    'Connection': 'keep-alive',
    'DNT': '1',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'downlink': '2.1',
    'ect': '4g',
    'rtt': '100',
    'sec-ch-ua': '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"10.0.0"',
}

def get_rating_data(ASIN):
    params = {
    'ie': 'UTF8',
    'asin': ASIN,
    'ref': 'acr_search__popover',
    'contextId': 'search',
    }
    proxy = random.choice(proxy_list)
    s_proxy_dict = {"http": proxy, 'https': proxy}

    response = requests.get(
        'https://www.amazon.cn/review/widgets/average-customer-review/popover/ref=acr_search__popover',
        params=params,
        headers=headers,
        proxies=s_proxy_dict
    )
    dom = lxml.html.fromstring(response.text)
    data_dict = {}
    data_dict['ASIN'] = ASIN
    stars = dom.xpath('//div[@role="progressbar"]/@aria-valuenow')

    for i in range(5,0,-1):
        data_dict[f'{i}_Stars']=stars[5-i]

    totalRatingCount = dom.xpath('//span[contains(@class,"totalRatingCount")]/text()')[0]
    data_dict["totalRatingCount"] = Clean(totalRatingCount.replace("买家评级",""))

    rating = dom.xpath('//span[contains(@data-hook,"acr-average")]/text()')[0]
    data_dict["rating"] = Clean(rating.replace("星，共 5 星",""))
    return data_dict

    

In [36]:
get_rating_data('B094VXYXBT')

{'ASIN': 'B094VXYXBT',
 '5_Stars': '52%',
 '4_Stars': '13%',
 '3_Stars': '0%',
 '2_Stars': '0%',
 '1_Stars': '35%',
 'totalRatingCount': '12',
 'rating': '3.5'}

In [60]:
from concurrent.futures import ThreadPoolExecutor
result =[]
with ThreadPoolExecutor(max_workers=12) as exe:
    exe.submit(get_rating_data,2)
        
    # Maps the method 'cube' with a list of values.
    result = exe.map(get_rating_data,products)

In [61]:
import pickle

with open('amazon_start_data.pickle', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)


TypeError: can't pickle generator objects

In [64]:
star_data = list(result)


In [1]:
import pandas as pd
df = pd.read_csv("H:\w_crawler\Amazon China Food\data\Amazon_China_Food.csv")

In [5]:
df['product_id'].unique()

array(['B07NFQH21P', 'B07WWHV4QW', 'B00442TL32', ..., 'B00YC0N3ZI',
       'B00R0KV73M', 'B002FTM3LI'], dtype=object)

In [13]:
with open(r'H:\w_crawler\Amazon China Food\asin_list.pickle', 'wb') as f:
    pickle.dump(list(df['product_id'].unique()), f)